In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report



In [3]:
df = pd.read_csv("C:\\Users\\Admin\\ML.COURSE\\IMDB Dataset.csv")
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [6]:
# Ініціалізуємо TF-IDF векторизатора и видалемо стоп-слова
nltk.download('stopwords')
stop_words = list(stopwords.words('english'))
tfidf_vectorizer = TfidfVectorizer(max_features=1000, stop_words=stop_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [7]:

tfidf_matrix = tfidf_vectorizer.fit_transform(df['review'])


In [8]:
# Трейн-тест спліт
X_train, X_temp, y_train, y_temp = train_test_split(tfidf_matrix, df['sentiment'], test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [9]:
# Навчемо класифікатор
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_train, y_train)

SVC(kernel='linear')

In [10]:
# Прогноз на тест-наборі
y_pred = svm_classifier.predict(X_test)
# Оцінемо точність
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.8669333333333333


In [11]:
# Зробимо класіфікейшн репорт
report = classification_report(y_test, y_pred)
print('Classification Report:\n', report)

Classification Report:
               precision    recall  f1-score   support

    negative       0.87      0.86      0.86      3722
    positive       0.86      0.88      0.87      3778

    accuracy                           0.87      7500
   macro avg       0.87      0.87      0.87      7500
weighted avg       0.87      0.87      0.87      7500



#### Тепер подивимося як працює моделька

In [20]:
import joblib
joblib.dump(svm_classifier, 'svm_model.pkl')
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')


['tfidf_vectorizer.pkl']

In [21]:
loaded_svm_model = joblib.load('svm_model.pkl')
loaded_tfidf_vectorizer = joblib.load('tfidf_vectorizer.pkl')

In [22]:
def classify_review(review_text):
    input_vector = loaded_tfidf_vectorizer.transform([review_text])
    prediction = loaded_svm_model.predict(input_vector)
    return prediction[0]


In [23]:
user_review = input("Введіть свій відгук: ")
prediction = classify_review(user_review)

Введіть свій відгук: This movie was a complete disappointment. The acting was terrible, the plot was confusing, and I couldn't wait for it to end. I wouldn't recommend it to anyone.


In [24]:
if prediction == 'positive':
    print("Відгук позитивний")
else:
    print("Відгук негативний")

Відгук негативний
